In [1]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import pandas as pd
import datetime
import locale
from data import df
import re
from unidecode import unidecode
from IPython.display import display
from pandas import DataFrame
import json

In [147]:
'''def cantidad_filmaciones_mes(mes: str):

    # Convert month to spanish
    mes = mes.lower().strip()

    # Verify if the month is  in the dict
    if mes in meses:
        mes_en_ingles = meses[mes]
        cantidad = len(df[df["release_date"].str.contains(mes_en_ingles, case=False)])



    #cantidad = len(df[df["release_date"].str.contains(mes, case=False)])

    if cantidad == 0:
        return f"No se encontraron películas estrenadas en el mes de {mes}"

    # Generate link to view the films for the given month
    link_html = f'<a href="/view_films/{mes}" target="_blank">Ver películas</a>'

    response = f"{cantidad} películas fueron estrenadas en el mes de {mes}"
    response += f"<br/>{link_html}"

    return HTMLResponse(content=response, status_code=200)'''

'def cantidad_filmaciones_mes(mes: str):\n\n    # Convert month to spanish\n    mes = mes.lower().strip()\n\n    # Verify if the month is  in the dict\n    if mes in meses:\n        mes_en_ingles = meses[mes]\n        cantidad = len(df[df["release_date"].str.contains(mes_en_ingles, case=False)])\n\n\n\n    #cantidad = len(df[df["release_date"].str.contains(mes, case=False)])\n\n    if cantidad == 0:\n        return f"No se encontraron películas estrenadas en el mes de {mes}"\n\n    # Generate link to view the films for the given month\n    link_html = f\'<a href="/view_films/{mes}" target="_blank">Ver películas</a>\'\n\n    response = f"{cantidad} películas fueron estrenadas en el mes de {mes}"\n    response += f"<br/>{link_html}"\n\n    return HTMLResponse(content=response, status_code=200)'

In [148]:
'''def total_films_day(dia: str):
    dia = dia.lower().strip()

    if dia in dias:
        dia_en_ingles = dias[dia]
        films = df[df["release_day"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains(dia_en_ingles, case=False)]

        if films.empty:
            return f"No se encontraron películas estrenadas en los días {dia}"

        link_html = f'<a href="/view_films_day/{dia}" target="_blank">Ver películas</a>'

        response = f"{len(films)} películas fueron estrenadas en los días {dia}"
        response += f"<br/>{link_html}"

        return HTMLResponse(content=response, status_code=200)
    else:
        return f"No se reconoce el día '{dia}'. Por favor, ingresa un día de la semana válido en español."'''

'def total_films_day(dia: str):\n    dia = dia.lower().strip()\n\n    if dia in dias:\n        dia_en_ingles = dias[dia]\n        films = df[df["release_day"].str.normalize(\'NFKD\').str.encode(\'ascii\', errors=\'ignore\').str.decode(\'utf-8\').str.lower().str.contains(dia_en_ingles, case=False)]\n\n        if films.empty:\n            return f"No se encontraron películas estrenadas en los días {dia}"\n\n        link_html = f\'<a href="/view_films_day/{dia}" target="_blank">Ver películas</a>\'\n\n        response = f"{len(films)} películas fueron estrenadas en los días {dia}"\n        response += f"<br/>{link_html}"\n\n        return HTMLResponse(content=response, status_code=200)\n    else:\n        return f"No se reconoce el día \'{dia}\'. Por favor, ingresa un día de la semana válido en español."'

In [149]:
'''def score_title(film_title: str):
    films = df[df["title"].str.lower() == film_title.lower()]
    if films.empty:
        return "No se encontró la filmación especificada."

    ordered_films = films.sort_values(by="release_year")

    table_html = "<table>"
    table_html += "<tr><th>Película</th><th>Año estreno</th><th>Score/Popularidad</th></tr>"

    for index, row in ordered_films.iterrows():
        title = row["title"]
        year_released = row["release_year"]
        score = row["popularity"]

        table_html += "<tr>"
        table_html += f"<td>{title}</td>"
        table_html += f"<td>{year_released}</td>"
        table_html += f"<td>{score}</td>"
        table_html += "</tr>"

    table_html += "</table>"

    return HTMLResponse(content=table_html, status_code=200)'''

'def score_title(film_title: str):\n    films = df[df["title"].str.lower() == film_title.lower()]\n    if films.empty:\n        return "No se encontró la filmación especificada."\n\n    ordered_films = films.sort_values(by="release_year")\n\n    table_html = "<table>"\n    table_html += "<tr><th>Película</th><th>Año estreno</th><th>Score/Popularidad</th></tr>"\n\n    for index, row in ordered_films.iterrows():\n        title = row["title"]\n        year_released = row["release_year"]\n        score = row["popularity"]\n\n        table_html += "<tr>"\n        table_html += f"<td>{title}</td>"\n        table_html += f"<td>{year_released}</td>"\n        table_html += f"<td>{score}</td>"\n        table_html += "</tr>"\n\n    table_html += "</table>"\n\n    return HTMLResponse(content=table_html, status_code=200)'

In [150]:
'''def votos_titulo(titulo_de_la_filmacion: str):
    filmaciones = df[df["title"].str.lower() == titulo_de_la_filmacion.lower()]
    
    if filmaciones.empty:
        return "No se encontró la filmación especificada."
    
    respuestas = []
    
    for _, row in filmaciones.iterrows():
        cantidad_votos = row["vote_count"]
        
        if cantidad_votos < 2000:
            continue  # Saltar esta película si no cumple con la cantidad mínima de votos
        
        titulo = row["title"]
        promedio_votos = row["vote_average"]
        respuesta = f"<tr><td>{titulo}</td><td>{cantidad_votos}</td><td>{promedio_votos}</td></tr>"
        respuestas.append(respuesta)
    
    if not respuestas:
        return f"<h1>No se encontraron filmaciones que cumplan con la cantidad mínima de votos requerida (2000 votos).</h1>"
    
    tabla_html = "<table style='border-collapse: collapse; width: 100%;'>"
    tabla_html += "<tr style='border-bottom: 1px solid #ddd;'><th style='padding: 8px; text-align: left;'>Título</th>"
    tabla_html += "<th style='padding: 8px; text-align: left;'>Cantidad de votos</th>"
    tabla_html += "<th style='padding: 8px; text-align: left;'>Valor promedio de votaciones</th></tr>"
    tabla_html += "".join(respuestas)
    tabla_html += "</table>"
    
    return f"<h1>Filmaciones con título '{titulo_de_la_filmacion}'</h1>{tabla_html}"'''


'def votos_titulo(titulo_de_la_filmacion: str):\n    filmaciones = df[df["title"].str.lower() == titulo_de_la_filmacion.lower()]\n    \n    if filmaciones.empty:\n        return "No se encontró la filmación especificada."\n    \n    respuestas = []\n    \n    for _, row in filmaciones.iterrows():\n        cantidad_votos = row["vote_count"]\n        \n        if cantidad_votos < 2000:\n            continue  # Saltar esta película si no cumple con la cantidad mínima de votos\n        \n        titulo = row["title"]\n        promedio_votos = row["vote_average"]\n        respuesta = f"<tr><td>{titulo}</td><td>{cantidad_votos}</td><td>{promedio_votos}</td></tr>"\n        respuestas.append(respuesta)\n    \n    if not respuestas:\n        return f"<h1>No se encontraron filmaciones que cumplan con la cantidad mínima de votos requerida (2000 votos).</h1>"\n    \n    tabla_html = "<table style=\'border-collapse: collapse; width: 100%;\'>"\n    tabla_html += "<tr style=\'border-bottom: 1px solid

In [2]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,countrie_name,iso_lang,lang_name,release_year,return,character,actor_id,actor_name,Director,release_day
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-October-30,373554033.0,81.0,Released,NaN,...,[United States of America],[en],[English],1995,12.451801,"[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,Monday
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-December-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,[United States of America],"[en, fr]","[English, Français]",1995,4.043035,"[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,Friday
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-December-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,[United States of America],[en],[English],1995,0.000000,"[Max Goldman, John Gustafson, Ariel Gustafson,...","[6837, 3151, 13567, 16757, 589, 16523, 7166]","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,Friday
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-December-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[United States of America],[en],[English],1995,5.090760,"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,Friday
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-February-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,[United States of America],[en],[English],1995,0.000000,"[George Banks, Nina Banks, Franck Eggelhoffer,...","[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,Friday


In [3]:
# Instance of FastApi
app = FastAPI()

In [153]:
meses = {
        "enero": "January",
        "febrero": "February",
        "marzo": "March",
        "abril": "April",
        "mayo": "May",
        "junio": "June",
        "julio": "July",
        "agosto": "August",
        "septiembre": "September",
        "octubre": "October",
        "noviembre": "November",
        "diciembre": "December"
    }

In [154]:
# 1 function number of films per month

@app.get("/cantidad_filmaciones_mes/{mes}")
def cantidad_filmaciones_mes(mes: str):
    # Convert month to Spanish
    mes = mes.lower().strip()

    # Verify if the month is in the dict
    if mes in meses:
        mes_en_ingles = meses[mes]
        cantidad = len(df[df["release_date"].str.contains(mes_en_ingles, case=False)])
    else:
        return {'mes': mes, 'cantidad': f"No se encontraron películas estrenadas en el mes de {mes}"}

    respuesta = f"{cantidad} películas fueron estrenadas en el mes de {mes}"

    return {'mes': mes, 'cantidad': respuesta}

In [155]:
cantidad_filmaciones_mes('mayo')

{'mes': 'mayo',
 'cantidad': '3339 películas fueron estrenadas en el mes de mayo'}

In [156]:
dias = {
        "lunes": "Monday",
        "martes": "Tuesday",
        "miércoles": "Wednesday",
        "jueves": "Thursday",
        "viernes": "Friday",
        "sábado": "Saturday",
        "domingo": "Sunday"
    }

In [157]:
@app.get("/cantidad_filmaciones_dia/{dia}")
def cantidad_filmaciones_dia(dia: str):
    dia = dia.lower().strip()

    if dia in dias:
        dia_en_ingles = dias[dia]
        films = df[df["release_day"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains(dia_en_ingles, case=False)]

        if films.empty:
            return {'dia': dia, 'cantidad': f"No se encontraron películas estrenadas en los días {dia}"}

        respuesta = f"{len(films)} películas fueron estrenadas en los días {dia}"

        return {'dia': dia, 'cantidad': respuesta}
    else:
        return {'dia': dia, 'cantidad': f"No se reconoce el día '{dia}'. Por favor, ingresa un día de la semana válido en español."}

In [158]:
cantidad_filmaciones_dia('jueves')

{'dia': 'jueves',
 'cantidad': '7527 películas fueron estrenadas en los días jueves'}

In [159]:
@app.get("/score_title/{film_title}")
def score_titulo(film_title: str):
    films = df[df["title"].str.lower() == film_title.lower()]
    if films.empty:
        return {'titulo': film_title, 'anio': None, 'popularidad': "No se encontró la filmación especificada."}

    ordered_films = films.sort_values(by="release_year")

    respuesta = []
    for index, row in ordered_films.iterrows():
        title = row["title"]
        year_released = row["release_year"]
        score = row["popularity"]
        respuesta.append({'titulo': title, 'anio': year_released, 'popularidad': score})

    return respuesta

In [160]:
score_titulo('cinderella')

[{'titulo': 'Cinderella', 'anio': 1899, 'popularidad': 1.301063},
 {'titulo': 'Cinderella', 'anio': 1914, 'popularidad': 0.361288},
 {'titulo': 'Cinderella', 'anio': 1947, 'popularidad': 0.486073},
 {'titulo': 'Cinderella', 'anio': 1950, 'popularidad': 14.436858},
 {'titulo': 'Cinderella', 'anio': 1957, 'popularidad': 0.209552},
 {'titulo': 'Cinderella', 'anio': 1997, 'popularidad': 2.102013},
 {'titulo': 'Cinderella', 'anio': 2000, 'popularidad': 1.085134},
 {'titulo': 'Cinderella', 'anio': 2002, 'popularidad': 2.154873},
 {'titulo': 'Cinderella', 'anio': 2011, 'popularidad': 0.987462},
 {'titulo': 'Cinderella', 'anio': 2012, 'popularidad': 0.803588},
 {'titulo': 'Cinderella', 'anio': 2015, 'popularidad': 19.801128}]

In [161]:
def score_title(film_title: str):
    films = df[df["title"].str.lower() == film_title.lower()]
    if films.empty:
        return "No se encontró la filmación especificada."

    ordered_films = films.sort_values(by="release_year")

    table_html = "<table>"
    table_html += "<tr><th>Película</th><th>Año estreno</th><th>Score/Popularidad</th></tr>"

    for index, row in ordered_films.iterrows():
        title = row["title"]
        year_released = row["release_year"]
        score = row["popularity"]

        table_html += "<tr>"
        table_html += f"<td>{title}</td>"
        table_html += f"<td>{year_released}</td>"
        table_html += f"<td>{score}</td>"
        table_html += "</tr>"

    table_html += "</table>"

    return HTMLResponse(content=table_html, status_code=200)

In [162]:
pd.set_option('display.max_columns', None)

In [163]:
def get_actor(actor_name):
    actor_films = df[df["actor_name"].str.contains(fr"\b{actor_name}\b", case=False, regex=True, na=False)]
    
    if actor_films.empty:
        return "No se encontró el actor especificado."

    film_count = len(actor_films)
    total_return = actor_films["return"].sum()
    average_return = total_return / film_count

    respuesta = f"El actor {actor_name} ha participado en {film_count} filmaciones. Ha conseguido un retorno total de {total_return} con un promedio de {average_return} por filmación."
    return respuesta


In [164]:
get_actor('woody allen')

'No se encontró el actor especificado.'

In [165]:
def get_director(director_name):
    director_movies = df[df['Director'].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]
    if director_movies.empty:
        return {"message": "No se encontró el director especificado."}
    film_count = len(director_movies)
    total_return = director_movies['return'].sum()
    answer = f"El director {director_name} ha dirigido {film_count} película(s). Con un ROI total de {total_return}"
    movies_data = director_movies[['Director', 'title', 'release_date', 'return', 'budget', 'revenue']]

    
    return {"answer": answer, "movies_data": movies_data}

In [166]:
get_director('woody')

{'answer': 'El director woody ha dirigido 49 película(s). Con un ROI total de 96.88784128640854',
 'movies_data':                                                 Director  \
 51                                           Woody Allen   
 343                                          Woody Allen   
 487                                          Woody Allen   
 1028                                         Woody Allen   
 1046                                         Woody Allen   
 1047                                         Woody Allen   
 1187                                         Woody Allen   
 1200                                         Woody Allen   
 1621                                         Woody Allen   
 1854                                         Woody Allen   
 1956                                         Woody Allen   
 2177                                         Woody Allen   
 2243                                         Woody Allen   
 2636                            

In [167]:
def get_director(director_name: str):
    # Filtrar las películas por director
    director_movies = df[df['Director'].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]

    if director_movies.empty:
        return "No se encontró el director especificado."

    film_count = len(director_movies)
    total_return = director_movies['return'].sum()
    answer = f"El director {director_name} ha dirigido {film_count} pelicula(s). Con un ROI total de {total_return}"

    # Generar la representación HTML de la tabla
    movies_table = director_movies[['Director', 'title', 'release_date', 'return', 'budget', 'revenue']]
    movies_table_html = movies_table.to_html()

    # Crear la respuesta HTML con el texto y la tabla generada
    response_content = f"<h1>{answer}</h1>{movies_table_html}"
    return HTMLResponse(content=response_content)

In [168]:
def get_director(director_name: str):
    # Filtrar las películas por director
    director_movies = df[df['Director'].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]

    if director_movies.empty:
        return "No se encontró el director especificado."

    film_count = len(director_movies)
    total_return = round(director_movies['return'].sum(), 2)
    
    movies_table = round(director_movies[['title', 'release_year', 'return', 'budget', 'revenue']], 2)
    


    return {'Director': director_name,
            'retorno_total_director': total_return,
            'total_peliculas': film_count,
            'peliculas': movies_table.to_dict(orient='records')}

In [169]:
get_director('michael moore')

{'Director': 'michael moore',
 'retorno_total_director': 74.27,
 'total_peliculas': 10,
 'peliculas': [{'title': 'Canadian Bacon',
   'release_year': 1995,
   'return': 0.02,
   'budget': 11000000,
   'revenue': 178104.0},
  {'title': 'The Big One',
   'release_year': 1997,
   'return': 0.0,
   'budget': 0,
   'revenue': 720074.0},
  {'title': 'Roger & Me',
   'release_year': 1989,
   'return': 41.91,
   'budget': 160000,
   'revenue': 6706368.0},
  {'title': 'Bowling for Columbine',
   'release_year': 2002,
   'return': 8.89,
   'budget': 4000000,
   'revenue': 35564473.0},
  {'title': 'Fahrenheit 9/11',
   'release_year': 2004,
   'return': 19.85,
   'budget': 6000000,
   'revenue': 119114517.0},
  {'title': 'Sicko',
   'release_year': 2007,
   'return': 2.73,
   'budget': 9000000,
   'revenue': 24538513.0},
  {'title': 'Slacker Uprising',
   'release_year': 2007,
   'return': 0.0,
   'budget': 0,
   'revenue': 0.0},
  {'title': 'Capitalism: A Love Story',
   'release_year': 2009,
  

In [170]:
def get_director(director_name):
    director_films = df[df["Director"].str.contains(fr"\b{director_name}\b", case=False, regex=True, na=False)]
    
    if director_films.empty:
        return "No se encontró el director especificado."

    films_info = director_films[["title", "release_date", "return", "budget", "revenue"]]
    respuesta = f"El director {director_name} ha dirigido las siguientes películas:\n\n"
    for _, row in films_info.iterrows():
        titulo = row["title"]
        fecha_lanzamiento = row["release_date"]
        retorno_individual = row["return"]
        costo = row["budget"]
        ganancia = row["revenue"]
        respuesta += f"Título: {titulo}\nFecha de lanzamiento: {fecha_lanzamiento}\nRetorno individual: {retorno_individual}\nCosto: {costo}\nGanancia: {ganancia}\n\n"

    return respuesta


In [171]:
get_director('steven spealberg')

'No se encontró el director especificado.'

In [172]:
@app.get("/votos_titulo/{titulo_de_la_filmacion}")
def votos_titulo(titulo_de_la_filmacion: str):
    filmaciones = df[df["title"].str.lower() == titulo_de_la_filmacion.lower()]
    
    if filmaciones.empty:
        return "No se encontró la filmación especificada."
    
    respuestas = []
    
    for _, row in filmaciones.iterrows():
        cantidad_votos = row["vote_count"]
        
        if cantidad_votos < 2000:
            continue  # Saltar esta película si no cumple con la cantidad mínima de votos
        
        titulo = row["title"]
        promedio_votos = row["vote_average"]
        respuesta = f"Título: {titulo}<br/>Cantidad de votos: {cantidad_votos}<br/>Valor promedio de votaciones: {promedio_votos}<br/><br/>"
        respuestas.append(respuesta)
    
    if not respuestas:
        return "No se encontraron filmaciones que cumplan con la cantidad mínima de votos requerida (2000 votos)."
    
    return "\n".join(respuestas)


In [173]:
votos_titulo('Cinderella')

'Título: Cinderella<br/>Cantidad de votos: 2426.0<br/>Valor promedio de votaciones: 6.7<br/><br/>'

In [175]:
# Transform the list columns OJO HACER ESTO EN EL DATA.PY PARA REVISAR COMO QUEDAN FUNCIONANDO LAS FUNCIONES DE LA API
import ast
columns_lists = ['countrie_name', 'iso_lang', 'lang_name', 'character', 'actor_name', 'genre', 'iso', 'companies_name']  # list of columns with lists

for columna in columns_lists:
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [176]:
df.iso

0                    [US]
1                    [US]
2                    [US]
3                    [US]
4                    [US]
               ...       
45384    [CA, DE, GB, US]
45385                [PH]
45386                [US]
45387                [RU]
45388                [GB]
Name: iso, Length: 45389, dtype: object

In [177]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'id_collection', 'collection', 'genre_id',
       'genre', 'companies_id', 'companies_name', 'iso', 'countrie_name',
       'iso_lang', 'lang_name', 'release_year', 'return', 'character',
       'actor_id', 'actor_name', 'Director', 'release_day'],
      dtype='object')

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Remove rows with missing or null values in relevant columns
df1 = df.drop(columns=['budget', 'id', 'revenue', 'release_date', 'status', 'runtime', 'actor_name', 'genre', 'character', 'collection', 'status', 'tagline', 'vote_count', 'id_collection', 'genre', 'companies_id', 'companies_name', 'countrie_name', 'lang_name', 'release_year', 'return', 'character', 'lang_name'])

# Convert actor and genre lists to strings for processing
df1['actor_id'] = df1['actor_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
df1['genre_id'] = df1['genre_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Create a term frequency matrix using CountVectorizer for relevant columns
vectorizer = CountVectorizer()
term_matrix = vectorizer.fit_transform(df1['genre_id'] + ' ' + df1['actor_id'] + ' ' + df1['popularity'].astype(str) + ' ' + df1['vote_average'].astype(str))

# Function to get movies similar to a given movie
def obtener_peliculas_similares(titulo, n=5):
    titulo = titulo.lower()
    indice_pelicula = df1[df1['title'].str.lower() == titulo].index
    if len(indice_pelicula) == 0:
        print('No se encontró la película:', titulo)
        return []

    indice_pelicula = indice_pelicula[0]
    vector_pelicula = term_matrix[indice_pelicula]
    similaridades = cosine_similarity(vector_pelicula, term_matrix)[0]
    indices_similares = similaridades.argsort()[::-1][1:n+1]  # Exclude the given movie

    # Sort the similar movies based on similarity score
    indices_similares_sorted = sorted(indices_similares, key=lambda x: similaridades[x], reverse=True)
    peliculas_similares = df1.iloc[indices_similares_sorted]['title'].tolist()
    return peliculas_similares

@app.get('/recomendacion/{titulo}')
def recomendacion(titulo: str):
    peliculas_recomendadas = obtener_peliculas_similares(titulo)
    return {'lista recomendada': peliculas_recomendadas}



In [7]:
recomendacion('Shine')

{'lista recomendada': ['The Recruit',
  'Sleepwalking',
  'Man of Tai Chi',
  'The Son of No One',
  "The Devil's Backbone"]}

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Remove rows with missing or null values in relevant columns
df1 = df.drop(columns=['budget', 'id', 'revenue', 'release_date', 'status', 'runtime', 'actor_name', 'genre', 'character', 'collection', 'status', 'tagline', 'vote_count', 'id_collection', 'genre', 'companies_id', 'companies_name', 'countrie_name', 'lang_name', 'release_year', 'return', 'character', 'lang_name'])


# Create a term frequency matrix using CountVectorizer for relevant columns
vectorizer = CountVectorizer()
term_matrix = vectorizer.fit_transform(df1['genre_id']+ ' ' + df1['actor_id'] + ' ' + df1['popularity'].astype(str) + ' ' + df1['vote_average'].astype(str))

# Function to get movies similar to a given movie
def obtener_peliculas_similares(titulo, n=5):
    titulo = titulo.lower()
    indice_pelicula = df1[df1['title'].str.lower() == titulo].index
    if len(indice_pelicula) == 0:
        return 'No se encontró la película, revisa si está bien escrita'

    indice_pelicula = indice_pelicula[0]
    vector_pelicula = term_matrix[indice_pelicula]
    similaridades = cosine_similarity(vector_pelicula, term_matrix)[0]
    indices_similares = similaridades.argsort()[::-1][1:n+1]  # Exclude the given movie

    # Sort the similar movies based on similarity score
    indices_similares_sorted = sorted(indices_similares, key=lambda x: similaridades[x], reverse=True)
    peliculas_similares = df1.iloc[indices_similares_sorted]['title'].tolist()
    return peliculas_similares

@app.get('/recomendacion/{titulo}')
def recomendacion(titulo: str):
    peliculas_recomendadas = obtener_peliculas_similares(titulo)
    return {'lista recomendada': peliculas_recomendadas}

In [30]:
recomendacion('hotel transilvanya')

{'lista recomendada': 'No se encontró la película, revisa si está bien escrita'}

In [4]:
df1 = df.drop(columns=['budget', 'id', 'revenue', 'release_date', 'status', 'runtime', 'actor_name', 'genre', 'character', 'collection', 'status', 'tagline', 'vote_count', 'id_collection', 'genre', 'companies_id', 'companies_name', 'countrie_name', 'lang_name', 'release_year', 'return', 'character', 'lang_name'])
df1

,original_language,overview,popularity,title,vote_average,genre_id,iso,iso_lang,actor_id,Director,release_day
0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Toy Story,7.7,"[16, 35, 10751]",[US],[en],"[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...",John Lasseter,Monday
1,en,When siblings Judy and Peter discover an encha...,17.015539,Jumanji,6.9,"[12, 14, 10751]",[US],"[en, fr]","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...",Joe Johnston,Friday
2,en,A family wedding reignites the ancient feud be...,11.712900,Grumpier Old Men,6.5,"[10749, 35]",[US],[en],"[6837, 3151, 13567, 16757, 589, 16523, 7166]",Howard Deutch,Friday
3,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Waiting to Exhale,6.1,"[35, 18, 10749]",[US],[en],"[8851, 9780, 18284, 51359, 66804, 352, 87118, ...",Forest Whitaker,Friday
4,en,Just when George Banks has recovered from his ...,8.387519,Father of the Bride Part II,5.7,[35],[US],[en],"[67773, 3092, 519, 70696, 59222, 18793, 14592,...",Charles Shyer,Friday
...,...,...,...,...,...,...,...,...,...,...,...
45384,en,"Yet another version of the classic epic, with ...",5.683753,Robin Hood,5.7,"[18, 28, 10749]","[CA, DE, GB, US]",[en],"[29459, 139, 18616, 920, 1924]",John Irvin,Monday
45385,tl,An artist struggles to finish his work while a...,0.178241,Century of Birthing,9.0,[18],[PH],[tl],"[1043186, 111636, 1204271, 278923, 1042953, 57...",Lav Diaz,Thursday
45386,en,"When one of her hits goes wrong, a professiona...",0.903007,Betrayal,3.8,"[28, 18, 53]",[US],[en],"[23764, 2059, 46277, 1736, 58646, 54649, 55270...",Mark L. Lester,Friday
45387,en,"In a small town live two brothers, one a minis...",0.003503,Satan Triumphant,0.0,[],[RU],[],"[544742, 1090923, 1136422, 1261758, 29199]",Yakov Protazanov,Sunday


In [5]:
# Function to transform the columns
def clean_column_values(df, column_name):
    df[column_name] = df[column_name].astype(str).str.replace('[', '', regex=False)
    df[column_name] = df[column_name].astype(str).str.replace(']', '', regex=False)
    df[column_name] = df[column_name].astype(str).str.replace("'", '', regex=False)
    return df

In [32]:
# Create a dictionary to store Director names and their corresponding codes
director_codes = {}
code_counter = 0

# Function to generate a code (integer) for a Director's name
def generate_director_code():
    global code_counter
    code_counter += 1
    return code_counter

# Generate codes for Directors and store them in the DataFrame
for index, row in df1.iterrows():
    director_name = row['Director']
    
    # Check if the Director's name already has a code
    if director_name in director_codes:
        code = director_codes[director_name]
    else:
        # Generate a new code for the Director's name
        code = generate_director_code()
        director_codes[director_name] = code
    
    # Assign the code to the Director_Code column for that row
    df1.at[index, 'Director_Code'] = code

# Convert the Director_Code column to integer type
df1['Director_Code'] = df1['Director_Code'].astype(int)

# Print the DataFrame to see the result
df1

,original_language,overview,popularity,title,vote_average,genre_id,iso,iso_lang,actor_id,Director,release_day,Director_Code
0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Toy Story,7.7,"[16, 35, 10751]",[US],[en],"[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...",John Lasseter,Monday,1
1,en,When siblings Judy and Peter discover an encha...,17.015539,Jumanji,6.9,"[12, 14, 10751]",[US],"[en, fr]","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...",Joe Johnston,Friday,2
2,en,A family wedding reignites the ancient feud be...,11.712900,Grumpier Old Men,6.5,"[10749, 35]",[US],[en],"[6837, 3151, 13567, 16757, 589, 16523, 7166]",Howard Deutch,Friday,3
3,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Waiting to Exhale,6.1,"[35, 18, 10749]",[US],[en],"[8851, 9780, 18284, 51359, 66804, 352, 87118, ...",Forest Whitaker,Friday,4
4,en,Just when George Banks has recovered from his ...,8.387519,Father of the Bride Part II,5.7,[35],[US],[en],"[67773, 3092, 519, 70696, 59222, 18793, 14592,...",Charles Shyer,Friday,5
...,...,...,...,...,...,...,...,...,...,...,...,...
45384,en,"Yet another version of the classic epic, with ...",5.683753,Robin Hood,5.7,"[18, 28, 10749]","[CA, DE, GB, US]",[en],"[29459, 139, 18616, 920, 1924]",John Irvin,Monday,397
45385,tl,An artist struggles to finish his work while a...,0.178241,Century of Birthing,9.0,[18],[PH],[tl],"[1043186, 111636, 1204271, 278923, 1042953, 57...",Lav Diaz,Thursday,11511
45386,en,"When one of her hits goes wrong, a professiona...",0.903007,Betrayal,3.8,"[28, 18, 53]",[US],[en],"[23764, 2059, 46277, 1736, 58646, 54649, 55270...",Mark L. Lester,Friday,1382
45387,en,"In a small town live two brothers, one a minis...",0.003503,Satan Triumphant,0.0,[],[RU],[],"[544742, 1090923, 1136422, 1261758, 29199]",Yakov Protazanov,Sunday,5776


In [33]:
df1.Director.value_counts()

Director
John Ford                   63
Michael Curtiz              61
Werner Herzog               52
Alfred Hitchcock            52
Georges Méliès              51
                            ..
Rob Meltzer                  1
Ignacio Ferreras             1
Dylan Mohan Gray             1
Spike Jonze, Lance Bangs     1
Daisy Asquith                1
Name: count, Length: 18829, dtype: int64

In [34]:
df1.Director_Code.value_counts()

Director_Code
186      835
866       63
701       61
697       52
504       52
        ... 
9478       1
9479       1
9483       1
9484       1
18830      1
Name: count, Length: 18830, dtype: int64